# Восстановление пропущенных пробелов в тексте через DP

## Теоретический принцип решения

Задача формулируется как сегментация строки без пробелов на слова.  
Каждое возможное разбиение строки имеет «стоимость», и цель алгоритма — найти последовательность разрезов с минимальной суммарной стоимостью.

---

### Подход

1. **Формализация задачи.**  
   Для строки длины *n* строим динамику: на позиции *i* выбираем оптимальный разрез, исходя из предыдущих позиций и функции стоимости токена.

2. **Функция стоимости слова.**  
   Для оценки «правдоподобности» токена используется частотная модель:  
   - словарные частоты (`wordfreq` для русского и английского);  
   - эвристики для коротких служебных слов, чисел, брендов, акронимов;  
   - штраф за OOV (out-of-vocabulary).

3. **Алгоритм поиска.**  
   Динамическое программирование с ограничением максимальной длины токена (обычно 18–24).  
   На каждом шаге проверяются все разрезы длиной ≤ L и выбирается минимальная стоимость.

4. **Оптимизация.**  
   - Ленивый кэш подстрок: каждый новый токен считается один раз, результат сохраняется.  
   - При повторных вызовах используется словарь `VOCAB`.  
   - Кэш сохраняется на диск и загружается при следующих запусках.  
   - Независимые строки обрабатываются параллельно.

5. **Валидация.**  
   Качество измеряется через F1-score по позициям пробелов: сравнение предсказанных индексов с эталонными.

---

## Достоинства подхода
- Лёгкий и воспроизводимый: CPU-дружественный, работает без GPU.  
- Гибкий: можно расширять словарь частот или добавлять эвристики.  
- Высокая устойчивость: не теряет символы, корректно работает даже на OOV.

## Ограничения
- Ассимптотика O(n·L): для очень длинных строк скорость падает.  
- Зависимость от качества словаря: редкие бренды и модели могут резаться неверно.  

## Загрузка библиотек

In [1]:
!pip -q install wordfreq joblib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00


In [9]:
import os, re, math, pickle
from functools import lru_cache
from typing import List, Iterable
from threading import Lock

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm.auto import tqdm
from wordfreq import zipf_frequency

In [10]:
pd.set_option("display.max_colwidth", 140)

# Пути к данным
TRAIN_PATH  = "train_dataset.csv"

STEPIC_PATH = "dataset_1937770_3.txt"           # тестовый файл
VOCAB_PATH  = "vocab_cache.pkl"                  # словарь-кэш подстрок

# Параметры алгоритма
MAX_TOK_LEN = 24
N_JOBS      = os.cpu_count() or 4

## Утилиты: парсинг, вставка пробелов, метрика

In [11]:
def parse_positions(x) -> List[int]:
    """Парсинг списка индексов пробелов из строки или списка"""
    if isinstance(x, list):
        return [int(v) for v in x]
    if isinstance(x, str):
        x = x.strip()
        return [] if not x else [int(v) for v in x.split()]
    return []

def insert_spaces(text_no_spaces: str, positions: Iterable[int]) -> str:
    """Вставить пробелы в строку по предсказанным позициям"""
    s = list(text_no_spaces)
    positions = sorted(int(p) for p in positions)
    for k, p in enumerate(positions):
        if 0 <= p < len(text_no_spaces) - 1:
            s.insert(p + 1 + k, " ")
    return "".join(s)

def f1_for_sets(pred: Iterable[int], true: Iterable[int]) -> float:
    """F1-метрика между предсказанными и истинными позициями"""
    P, T = set(pred), set(true)
    if not P and not T:
        return 1.0
    tp = len(P & T)
    if tp == 0:
        return 0.0
    prec = tp / (len(P) if P else 1)
    rec  = tp / (len(T) if T else 1)
    return 2 * prec * rec / (prec + rec)

## Функция стоимости слова + ленивый кэш VOCAB

In [12]:
# Частые короткие служебные слова
RU_SHORT = {
    "и","в","во","на","но","к","с","со","о","об","от","у","по","до",
    "из","за","для","без","при","над","под","я","мы","он","она","оно",
    "вы","они","ли","же","то","или","как","есть","нет","да","не"
}

# Небольшой набор брендов и технических слов
BRANDS = {
    "samsung","xiaomi","huawei","philips","dyson","iphone","ipad","ipod",
    "playstation","ps5","ps4","xbox","honor","msi","asus","acer","lenovo","dell",
    "canon","bosch","oppo","tcl","toshiba","lg","adidas","nike","pixel","rtx","gtx",
    "lego","nvidia","hp","hdd","ssd","jbl","sony","nikon"
}

RE_NUMERIC_UNIT = re.compile(r"^(\d+([.,]\d+)?)([a-zA-Zа-яА-Я]{0,6})$")
RE_UPPER_ALNUM  = re.compile(r"^[A-Z0-9]{2,}$")

@lru_cache(maxsize=300_000)
def max_zipf(w: str) -> float:
    """Максимальная частота слова по словарям (ru/en)"""
    if not w:
        return 0.0
    cands = {w, w.lower(), w.title()}
    vals = []
    for x in cands:
        vals.append(zipf_frequency(x, "ru", wordlist="large"))
        vals.append(zipf_frequency(x, "en", wordlist="large"))
    return max(vals)

@lru_cache(maxsize=300_000)
def compute_cost_raw(w: str) -> float:
    """Базовая функция стоимости слова"""
    if not w:
        return 1e6
    wl = w.lower()
    if RE_NUMERIC_UNIT.match(wl):
        return 2.0 + 0.05 * max(0, len(w) - 2)
    if wl in BRANDS:
        return 2.0 + 0.03 * max(0, len(w) - 4)
    if wl in RU_SHORT:
        return 1.5
    if RE_UPPER_ALNUM.match(w):
        return 2.2 + 0.06 * max(0, len(w) - 2)
    z = max_zipf(w)
    if z > 0:
        return max(0.5, 7.0 - z)
    return 4.0 + 0.55 * max(0, len(w) - 2)

# Ленивый кэш подстрок
VOCAB = {}
if os.path.exists(VOCAB_PATH):
    try:
        with open(VOCAB_PATH, "rb") as f:
            VOCAB = pickle.load(f)
        print(f"VOCAB загружен: {len(VOCAB)} подстрок")
    except Exception as e:
        print("Не удалось загрузить VOCAB, начинаю с пустого. Причина:", repr(e))
        VOCAB = {}
else:
    print("VOCAB не найден — начнём с пустого.")

def word_cost(w: str) -> float:
    """Стоимость слова с использованием кэша"""
    c = VOCAB.get(w)
    if c is not None:
        return c
    c = compute_cost_raw(w)
    VOCAB[w] = c
    return c

VOCAB загружен: 4966654 подстрок


## DP-сегментация текста

In [13]:
def dp_segment(text: str, max_tok_len: int = MAX_TOK_LEN) -> List[str]:
    """Разбивает строку без пробелов на токены с минимальной суммарной стоимостью"""
    n = len(text)
    if n == 0:
        return []

    dp_cost = [math.inf] * (n + 1)
    backptr = [-1] * (n + 1)
    dp_cost[0] = 0.0

    for i in range(1, n + 1):
        j_start = max(0, i - max_tok_len)
        best_c, best_j = math.inf, -1
        # проверяем все варианты разреза длиной до max_tok_len
        for j in range(i - 1, j_start - 1, -1):
            w = text[j:i]
            c = dp_cost[j] + word_cost(w)
            if c < best_c:
                best_c, best_j = c, j
        dp_cost[i] = best_c
        backptr[i] = best_j

    # восстановление последовательности токенов
    toks, i = [], n
    while i > 0 and backptr[i] >= 0:
        j = backptr[i]
        toks.append(text[j:i])
        i = j
    toks.reverse()
    return toks

def tokens_to_positions(tokens: List[str]) -> List[int]:
    """Преобразует список токенов в список позиций пробелов"""
    pos, cur = [], -1
    for t in tokens[:-1]:
        cur += len(t)
        pos.append(cur)
    return pos

def infer_one(s: str) -> List[int]:
    """Инференс для одной строки"""
    toks = dp_segment(s)
    return tokens_to_positions(toks)

## Валидация на 10% Wildberries

In [14]:
train_df = pd.read_csv(TRAIN_PATH)
train_df["target_positions"] = train_df["target_positions"].apply(parse_positions)
train_df = train_df.dropna(subset=["input_text"]).reset_index(drop=True)

# 10% выборка (детерминированно)
rng = np.random.default_rng(42)
k = max(1, int(len(train_df) * 0.10))
val_idx = rng.choice(len(train_df), size=k, replace=False)
val_df = train_df.iloc[val_idx].copy().reset_index(drop=True)

# Инференс на валидации (параллельно потоками)
texts = val_df["input_text"].astype(str).tolist()
pred_positions = Parallel(n_jobs=N_JOBS, prefer="threads")(
    delayed(infer_one)(s) for s in tqdm(texts, desc="Validate")
)

# Подсчёт F1
f1_list = [f1_for_sets(p, t) for p, t in zip(pred_positions, val_df["target_positions"])]
print("Средний F1 на 10% Wild:", float(np.mean(f1_list)))

Validate:   0%|          | 0/20834 [00:00<?, ?it/s]

Средний F1 на 10% Wild: 0.7842403810575063


In [15]:
# Примеры для проверки
val_df = val_df.copy()
val_df["predicted_positions"] = pred_positions
val_df["spaced_pred"] = [insert_spaces(s, pos) for s, pos in zip(val_df["input_text"], val_df["predicted_positions"])]
val_df["spaced_true"] = [insert_spaces(s, pos) for s, pos in zip(val_df["input_text"], val_df["target_positions"])]
display(val_df.sample(min(10, len(val_df)), random_state=1)[
    ["spaced_pred","spaced_true","predicted_positions","target_positions"]
])

,spaced_pred,spaced_true,predicted_positions,target_positions
18726,Халат из хлопкового велюраимитацияштроксас карманами наша левом воротнике принт,Халат из хлопкового велюра имитация штрокса с карманами на шалевом воротнике принт,"[4, 6, 16, 38, 47, 51, 56, 65]","[4, 6, 16, 22, 30, 37, 38, 47, 49, 56, 65]"
18778,1510911,151091 1,[],[5]
1295,Шкатулка токарная Традиционная городецкая роспись,Шкатулка токарная Традиционная городецкая роспись,"[7, 15, 27, 37]","[7, 15, 27, 37]"
18412,Картридж NVP совместимый NVCF352A Yellow для Color LaserJet ProM 176nM 177fw 1000HP130A,Картридж NVP совместимый NV CF352A Yellow для Color LaserJet Pro M176n M177fw 1000 HP 130A,"[7, 10, 21, 29, 35, 38, 43, 51, 55, 60, 65]","[7, 10, 21, 23, 29, 35, 38, 43, 51, 54, 59, 65, 69, 71]"
18036,В наборе 6разных цветов мишуры Длина каждой мишуры 2метра диаметр 9смшири на бахромы 08см,В наборе 6 разных цветов мишуры Длина каждой мишуры 2 метра диаметр 9 см ширина бахромы 0 8 см,"[0, 6, 13, 19, 25, 30, 36, 42, 48, 55, 62, 64, 71]","[0, 6, 7, 13, 19, 25, 30, 36, 42, 43, 48, 55, 56, 58, 64, 71, 72, 73]"
15877,1614531,161453 1,[],[5]
16288,Силиконовый чехол для Xiaomi Red m i 7Защитн а я накладка бампердляСяомиРедми7,Силиконовый чехол для Xiaomi Redmi 7 Защитная накладка бампер для Сяоми Редми 7,"[10, 15, 18, 24, 27, 28, 29, 36, 37, 38, 46]","[10, 15, 18, 24, 29, 30, 38, 46, 52, 55, 60, 65]"
20531,Паштет Нежный с мясом кролика Масса нет то 90грамм,Паштет Нежный с мясом кролика Масса нетто 90 грамм,"[5, 11, 12, 17, 24, 29, 32, 34]","[5, 11, 12, 17, 24, 29, 34, 36]"
7549,Платье на а тласномподкладесошлейфом,Платье на атласном подкладе со шлейфом,"[5, 7, 8]","[5, 7, 15, 23, 25]"
11977,Жакетвязаныйс манжетами Силуэт приталенный Отделка мех финского песца под со боль,Жакет вязаный с манжетами Силуэт приталенный Отделка мех финского песца под соболь,"[12, 21, 27, 38, 45, 48, 56, 61, 64, 66]","[4, 11, 12, 21, 27, 38, 45, 48, 56, 61, 64]"


In [16]:
# Сохраняем обновлённый словарь
with open(VOCAB_PATH, "wb") as f:
    pickle.dump(VOCAB, f)
print(f"VOCAB сохранён: {VOCAB_PATH}, размер {len(VOCAB)}")

VOCAB сохранён: vocab_cache.pkl, размер 15347076


## Тестирование на Stepik файле и формирование submission

In [17]:
# Перечитаем словарь
if os.path.exists(VOCAB_PATH):
    with open(VOCAB_PATH, "rb") as f:
        VOCAB = pickle.load(f)
    print(f"VOCAB подгружен: {len(VOCAB)} подстрок")

# Загружаем Stepik файл
task_data = pd.read_csv(
    STEPIC_PATH,
    header=0,
    names=["id", "text_no_spaces"],
    dtype={"id": "int64", "text_no_spaces": "string"},
    usecols=[0,1],
    engine="python"
)

texts = task_data["text_no_spaces"].astype(str).tolist()

# Инференс (параллельно потоками)
pred_positions = Parallel(n_jobs=N_JOBS, prefer="threads")(
    delayed(infer_one)(s) for s in tqdm(texts, desc="Stepik inference")
)

# Формируем submission
def fmt(pos):
  return "[" + ", ".join(map(str, [p+1 for p in pos])) + "]" if pos else "[]"

submission = pd.DataFrame({
    "id": task_data["id"].astype("int64"),
    "predicted_positions": [fmt(p) for p in pred_positions]
})

SUBMIT_PATH = "submission.csv"
submission.to_csv(SUBMIT_PATH, index=False, encoding="utf-8", lineterminator="\n")
print("Сохранено:", SUBMIT_PATH)
print(submission.head())

VOCAB подгружен: 15347076 подстрок


Stepik inference:   0%|          | 0/1005 [00:00<?, ?it/s]

Сохранено: submission.csv
   id  predicted_positions
0   0              [5, 10]
1   1            [3, 6, 7]
2   2  [4, 12, 13, 20, 21]
3   3          [5, 10, 18]
4   4              [5, 10]


In [18]:
# Берём первые 10 примеров с текстом и предсказаниями
ex = task_data.merge(submission, on="id", how="left").head(10).copy()

# Парсим список позиций из строки "[5, 10, 17]" -> [5, 10, 17]
ex["gap_positions"] = ex["predicted_positions"].apply(
    lambda s: [int(x) for x in re.findall(r"\d+", str(s))]
)

# Для визуализации пробелов конвертируем «щели» в индексы букв: p -> p-1
def gaps_to_left_indices(gaps, n):
    return [p-1 for p in gaps if 1 <= p <= n-1]

ex["spaced_pred"] = [
    insert_spaces(s, gaps_to_left_indices(g, len(s)))
    for s, g in zip(ex["text_no_spaces"], ex["gap_positions"])
]

display(ex[["id", "text_no_spaces", "predicted_positions", "spaced_pred"]])

,id,text_no_spaces,predicted_positions,spaced_pred
0,0,куплюайфон14про,"[5, 10]",куплю айфон 14про
1,1,ищудомвПодмосковье,"[3, 6, 7]",ищу дом в Подмосковье
2,2,сдаюквартирусмебельюитехникой,"[4, 12, 13, 20, 21]",сдаю квартиру с мебелью и техникой
3,3,новыйдивандоставканедорого,"[5, 10, 18]",новый диван доставка недорого
4,4,отдамдаромкошку,"[5, 10]",отдам даром кошку
5,5,работавМосквеудаленно,"[6, 7, 13]",работа в Москве удаленно
6,6,куплютелевизорPhilips,"[5, 14]",куплю телевизор Philips
7,7,ищугрузчиковдляпереезда,"[3, 12, 15]",ищу грузчиков для переезда
8,8,ремонтквартирподключ,"[6, 13, 16]",ремонт квартир под ключ
9,9,куплюноутбукHP,"[5, 12]",куплю ноутбук HP


In [12]:
# Обновляем и сохраняем VOCAB
with open(VOCAB_PATH, "wb") as f:
    pickle.dump(VOCAB, f)
print(f"VOCAB сохранён после Stepik: {VOCAB_PATH}, размер {len(VOCAB)}")

VOCAB сохранён после Stepik: vocab_cache.pkl, размер 3400531
